In [1]:
import pandas as pd
import motmetrics as mm

gt_file = r"C:\Users\rouat\Desktop\final\mot\test\gt\gt.txt"
res_file = r"C:\Users\rouat\Desktop\final\end_to_end_pipeline\tracking_results\tracking_results_yolo.txt"

def load_data_to_dict(file_path, is_gt=False):
    df = pd.read_csv(file_path, header=None)
    
    if is_gt:
        df.columns = ['frame','id','x','y','w','h','conf','cls','vis']
        df = df[df['conf'] == 1]   
    else:
        df.columns = ['frame','id','x','y','w','h','conf','cls']
    
    data_dict = {}
    for _, row in df.iterrows():
        frame_id = int(row['frame'])
        obj_id = int(row['id'])
        bbox = (row['x'], row['y'], row['w'], row['h'])
        data_dict.setdefault(frame_id, []).append((obj_id, bbox))
    return data_dict

gt_data = load_data_to_dict(gt_file, is_gt=True)
res_data = load_data_to_dict(res_file, is_gt=False)

acc = mm.MOTAccumulator(auto_id=True)

all_frames = sorted(set(gt_data.keys()) | set(res_data.keys()))
for frame in all_frames:
    gt_objs = gt_data.get(frame, [])
    res_objs = res_data.get(frame, [])

    gt_ids, gt_boxes = zip(*gt_objs) if gt_objs else ([], [])
    res_ids, res_boxes = zip(*res_objs) if res_objs else ([], [])

    if gt_boxes and res_boxes:
        iou_matrix = mm.distances.iou_matrix(gt_boxes, res_boxes, max_iou=0.5)
        distances = 1 - iou_matrix
    else:
        distances = []

    acc.update(gt_ids, res_ids, distances)

mh = mm.metrics.create()
metrics = ['mota', 'motp', 'idf1', 'precision', 'recall',
           'num_switches', 'num_false_positives', 'num_misses', 'num_fragmentations']
summary = mh.compute(acc, metrics=metrics, name='eval')
results = summary.loc['eval']

# Résultats lisibles
print("\n=== Résultats d'évaluation du tracking ===")
print(f"MOTA            : {results['mota']:.2%}")
print(f"MOTP            : {results['motp']:.2%}")
print(f"IDF1            : {results['idf1']:.2%}")
print(f"Précision       : {results['precision']:.2%}")
print(f"Rappel          : {results['recall']:.2%}")
print(f"ID Switches     : {int(results['num_switches'])}")
print(f"Faux positifs   : {int(results['num_false_positives'])}")
print(f"Misses          : {int(results['num_misses'])}")
print(f"Fragmentations  : {int(results['num_fragmentations'])}")



=== Résultats d'évaluation du tracking ===
MOTA            : 75.51%
MOTP            : 79.66%
IDF1            : 30.89%
Précision       : 95.12%
Rappel          : 81.94%
ID Switches     : 257
Faux positifs   : 487
Misses          : 2091
Fragmentations  : 327
